In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import re
import nltk.data
nltk.download()
from nltk.corpus import stopwords
from gensim.models import word2vec
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [22]:
# Read data from files 
path_raw = os.getcwd().replace('notebooks', 'data/raw/')

train = pd.read_csv(path_raw + "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

test = pd.read_csv(path_raw + "testData.tsv", header=0, delimiter="\t", quoting=3 )

unlabeled_train = pd.read_csv(path_raw + "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

In [23]:
# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" % (train["review"].size,  
test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [24]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().decode('utf-8').split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [25]:
# for sentence splitting
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [26]:
raw_sentences = tokenizer.tokenize(train["review"][0].strip())

In [27]:
print(raw_sentences[0])
print(raw_sentences[1])
print(raw_sentences[2])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again.
Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent.
Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released.


In [30]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.decode('utf-8').strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [31]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/bs4/__init__.py:294: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP

In [32]:
print("Number os setences")
print(len(sentences))
print("First setence")
print(sentences[0])
print("Second setence")
print(sentences[1])

Number os setences
795538
First setence
[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']
Second setence
[u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'was', u'really', u'cool', u'in', u'the', u'eighties', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent']


### Word2Vec

In [33]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)


2019-12-04 18:17:16,082 : INFO : collecting all words and their counts
2019-12-04 18:17:16,084 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-04 18:17:16,183 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types


Training model...


2019-12-04 18:17:16,281 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-12-04 18:17:16,383 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2019-12-04 18:17:16,480 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-12-04 18:17:16,576 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-12-04 18:17:16,675 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-12-04 18:17:16,773 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-12-04 18:17:16,867 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-12-04 18:17:16,964 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-12-04 18:17:17,061 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50

2019-12-04 18:17:23,447 : INFO : PROGRESS: at sentence #740000, processed 16552903 words, keeping 119668 word types
2019-12-04 18:17:23,564 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2019-12-04 18:17:23,680 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2019-12-04 18:17:23,796 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2019-12-04 18:17:23,912 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2019-12-04 18:17:24,025 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2019-12-04 18:17:24,085 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2019-12-04 18:17:24,086 : INFO : Loading a fresh vocabulary
2019-12-04 18:17:24,273 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2019

2019-12-04 18:18:12,471 : INFO : EPOCH 4 - PROGRESS: at 90.82% examples, 1048322 words/s, in_qsize 7, out_qsize 0
2019-12-04 18:18:13,479 : INFO : EPOCH 4 - PROGRESS: at 99.05% examples, 1047940 words/s, in_qsize 7, out_qsize 0
2019-12-04 18:18:13,570 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-12-04 18:18:13,578 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-12-04 18:18:13,586 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-12-04 18:18:13,589 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-12-04 18:18:13,590 : INFO : EPOCH - 4 : training on 17798082 raw words (12749873 effective words) took 12.2s, 1048323 effective words/s
2019-12-04 18:18:14,596 : INFO : EPOCH 5 - PROGRESS: at 8.16% examples, 1034613 words/s, in_qsize 8, out_qsize 0
2019-12-04 18:18:15,606 : INFO : EPOCH 5 - PROGRESS: at 16.44% examples, 1034485 words/s, in_qsize 7, out_qsize 0
2019-12-04 18:18:16,614 : INFO : EPOCH 5

In [34]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
path_raw = os.getcwd().replace('notebooks', 'models/')
model_name = path_raw + "300features_40minwords_10context"
model.save(model_name)

2019-12-04 18:18:26,140 : INFO : precomputing L2-norms of word weight vectors
2019-12-04 18:18:26,197 : INFO : saving Word2Vec object under /Users/loliveira/Documents/mygit/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context, separately None
2019-12-04 18:18:26,199 : INFO : not storing attribute vectors_norm
2019-12-04 18:18:26,201 : INFO : not storing attribute cum_table
2019-12-04 18:18:26,552 : INFO : saved /Users/loliveira/Documents/mygit/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context


In [35]:
#The "doesnt_match" function will try to deduce which word in a set is most dissimilar from the other
model.doesnt_match("france england germany berlin".split())

/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  
/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'berlin'

In [36]:
model.most_similar("man")

/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'woman', 0.6253253221511841),
 (u'lady', 0.6056747436523438),
 (u'lad', 0.5889188051223755),
 (u'monk', 0.5321874022483826),
 (u'men', 0.5245215892791748),
 (u'farmer', 0.5239187479019165),
 (u'businessman', 0.5186100602149963),
 (u'guy', 0.5137829780578613),
 (u'person', 0.5133359432220459),
 (u'millionaire', 0.5105389356613159)]

In [37]:
model.most_similar("awful")

/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'terrible', 0.7638652324676514),
 (u'atrocious', 0.7386058568954468),
 (u'horrible', 0.727492094039917),
 (u'abysmal', 0.7152676582336426),
 (u'dreadful', 0.7033779621124268),
 (u'horrid', 0.6975457072257996),
 (u'appalling', 0.6716896295547485),
 (u'horrendous', 0.6714738607406616),
 (u'lousy', 0.6360851526260376),
 (u'amateurish', 0.6360642910003662)]

In [38]:
from gensim.models import Word2Vec
model = Word2Vec.load(path_raw + "300features_40minwords_10context")

2019-12-04 18:18:26,641 : INFO : loading Word2Vec object from /Users/loliveira/Documents/mygit/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context
2019-12-04 18:18:26,755 : INFO : loading vocabulary recursively from /Users/loliveira/Documents/mygit/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context.vocabulary.* with mmap=None
2019-12-04 18:18:26,756 : INFO : loading wv recursively from /Users/loliveira/Documents/mygit/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context.wv.* with mmap=None
2019-12-04 18:18:26,758 : INFO : setting ignored attribute vectors_norm to None
2019-12-04 18:18:26,759 : INFO : loading trainables recursively from /Users/loliveira/Documents/mygit/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context.trainables.* with mmap=None
2019-12-04 18:18:26,761 : INFO : setting ignored attribute cum_table to None
2019-12-04 18:18:26,762 : INFO : loaded /Users/loliveira/Documents/mygit/kaggle-bag-of-words-popcorn/m

In [39]:
type(model.cum_table)


/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [40]:
model["flower"]

/Users/loliveira/miniconda3/envs/kaggle/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.00345678, -0.06302812, -0.13045566,  0.07635592,  0.02630942,
        0.02476116,  0.11651225,  0.01561824,  0.05033286,  0.05265347,
       -0.09409257, -0.14666888,  0.01195963,  0.12423845, -0.05012137,
       -0.00841881,  0.03761879,  0.01233195, -0.03428629, -0.13225643,
       -0.09181557, -0.03417945,  0.11022355,  0.04555974, -0.02662971,
       -0.00642254,  0.00089743, -0.03614795,  0.00335321, -0.00587334,
        0.02567563,  0.0125669 , -0.03779757,  0.00347613, -0.06679216,
        0.01590004,  0.07708862,  0.00176816,  0.0426693 ,  0.00399242,
       -0.01441719,  0.09620564, -0.02127444, -0.04582051,  0.03190284,
       -0.04550701, -0.0604981 , -0.01059696, -0.08915752,  0.13426977,
       -0.01754344,  0.06654104,  0.09395356, -0.0361356 , -0.05346743,
        0.04667037,  0.03068818, -0.02520963, -0.11137424,  0.0090273 ,
       -0.13961083,  0.00589716,  0.0513806 , -0.0604302 , -0.03605963,
        0.06707609, -0.06565899,  0.02196912,  0.1239398 , -0.02

### One challenge with the IMDB dataset is the variable-length reviews. We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

In [41]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs
